In [57]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [58]:
import sys
import os

In [59]:
import re
import inspect

In [60]:
sys.path.append("../scripts/")

In [61]:
import json

In [62]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [63]:
from transformers import Seq2SeqTrainer

In [64]:
from transformers.models.led.modeling_led import LEDModel

In [65]:
from tqdm import tqdm
import torch
from datasets import load_dataset, load_metric
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from dataset import CitationTextGenerationDataset, CitationParagraphGenerationDataset
from torch.utils.data import Dataset, DataLoader

In [66]:
from matplotlib import pyplot as plt
import seaborn as sns
plt.rcParams["figure.figsize"] = (10, 8)
sns.set()



In [67]:
device = "cuda"

In [68]:
max_input_length = 16384
max_output_length = 1024

In [69]:
def process_data_to_model_inputs(batch, special_tokens=['[Dominant]', '[Reference]']):
    # tokenize the inputs and labels
    
    additional_special_tokens_lookup = {token: idx for token, idx in zip(tokenizer.additional_special_tokens, tokenizer.additional_special_tokens_ids)}
    special_token_ids = set([additional_special_tokens_lookup[token] for token in special_tokens])
    special_token_ids.add(tokenizer.mask_token_id)
    
    inputs = tokenizer(
        batch["source"],
        padding="max_length",
        truncation=True,
        max_length=max_input_length,
        add_special_tokens=True 
    )
    outputs = tokenizer(
        batch["target"],
        padding="max_length",
        truncation=True,
        max_length=max_output_length,
        add_special_tokens=True 
    )

    batch["input_ids"] = inputs.input_ids
    batch["attention_mask"] = inputs.attention_mask

    # create 0 global_attention_mask lists
    batch["global_attention_mask"] = len(batch["input_ids"]) * [
        [0 for _ in range(len(batch["input_ids"][0]))]
    ]

    # since above lists are references, the following line changes the 0 index for all samples
    for i_batch in range(len(batch["input_ids"])):
        for i_token in range(len(batch["input_ids"][0])):
            if batch["input_ids"][i_batch][i_token] in special_token_ids:
                batch["global_attention_mask"][i_batch][i_token] = 1
            
    batch["labels"] = outputs.input_ids

    # We have to make sure that the PAD token is ignored
    batch["labels"] = [
        [-100 if token == tokenizer.pad_token_id else token for token in labels]
        for labels in batch["labels"]
    ]
    return batch

In [15]:
path = "/home/bxm200000/models/led_paragraph_generation_v3/checkpoint-29000/"

In [47]:
tokenizer = AutoTokenizer.from_pretrained(path)
special_tokens = ['<doc>','</doc>', '[BOS]', '[Dominant]', '[Reference]', '[B_Dominant]',  '[E_Dominant]', '[B_Reference]', '[E_Reference]', '<context>', '</context>']
additional_special_tokens = {'additional_special_tokens': special_tokens}
tokenizer.add_special_tokens(additional_special_tokens)

0

In [27]:
torch.cuda.empty_cache()

In [48]:
model = AutoModelForSeq2SeqLM.from_pretrained(
    path
)

In [49]:
model = model.to(device).half()
model.eval()

LEDForConditionalGeneration(
  (led): LEDModel(
    (shared): Embedding(50276, 768, padding_idx=1)
    (encoder): LEDEncoder(
      (embed_tokens): Embedding(50276, 768, padding_idx=1)
      (embed_positions): LEDLearnedPositionalEmbedding(16384, 768, padding_idx=1)
      (layers): ModuleList(
        (0): LEDEncoderLayer(
          (self_attn): LEDEncoderAttention(
            (longformer_self_attn): LEDEncoderSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (query_global): Linear(in_features=768, out_features=768, bias=True)
              (key_global): Linear(in_features=768, out_features=768, bias=True)
              (value_global): Linear(in_features=768, out_features=768, bias=True)
            )
            (output): Linear(in_features=768, out_features=768, bias=True)
        

In [ ]:
val_set = CitationParagraphGenerationDataset(
    "/home/data/XiangciLi/CORWA/annotated_test/", 
    tokenizer, 
    MAX_SENT_LEN = max_input_length,
    related_work_path='/home/data/XiangciLi/20200705v1/acl/selected_related_work.jsonl',
    cited_metadata_path='/home/data/XiangciLi/20200705v1/acl/selected_cited_metadata.jsonl',
    cited_paper_path="/home/data/XiangciLi/20200705v1/acl/selected_cited_pdf_parses.jsonl",
    citing_paper_path="/home/data/XiangciLi/20200705v1/acl/selected_pdf_parses.jsonl",
    skip_no_citations = True,
    context_sep_flag=True
)

 18%|█▊        | 64/362 [00:03<00:11, 26.74it/s]

In [70]:
def run_model(batch, model):
    processed_batch = process_data_to_model_inputs(batch, special_tokens=['[Dominant]', '[Reference]'])
    processed_batch_cuda = {}
    for key in ["input_ids", "attention_mask", "global_attention_mask", "labels"]:
        processed_batch_cuda[key] = torch.tensor(processed_batch[key]).to(device)
    predicted_abstract_ids = model.generate(
        processed_batch_cuda["input_ids"], 
        attention_mask=processed_batch_cuda["attention_mask"], 
        global_attention_mask=processed_batch_cuda["global_attention_mask"]
    )
    out = tokenizer.batch_decode(predicted_abstract_ids, skip_special_tokens=False)
    target = batch["target"]
    return out, target

In [ ]:
# for batch in tqdm(DataLoader(val_set, batch_size = 1, shuffle=False)):
#     if '[Dominant]' in batch["source"][0]:
#         out, target = run_model(batch, model)
#         print("Dominant: ", out)
#         batch["source"][0] = batch["source"][0].replace("[Dominant]", "[Reference]")
#         out, target = run_model(batch, model)
#         print("Reference: ", out)
#         print("Dominant label: ", target)
#         print()
#     elif '[Reference]' in batch["source"][0]:
#         out, target = run_model(batch, model)
#         print("Reference: ", out)
#         batch["source"][0] = batch["source"][0].replace("[Reference]", "[Dominant]")
#         out, target = run_model(batch, model)
#         print("Dominant: ", out)
#         print("Reference label: ", target)
#         print()

In [71]:
def get_citations(src):
    """Get citations given source content"""
    all_citations = []
    for cite_data in src.split("[B_Reference]")[1:]:

        all_citations.append(cite_data.split("</s>")[0].strip())

    for cite_data in src.split("[B_Dominant]")[1:]:

        all_citations.append(cite_data.split("</s>")[0].strip())
    
    return all_citations

In [ ]:
accumulated_data = []

#reference_predicted = []
#reference_reference = []
#dominant_predicted = []
#dominant_reference = []
for batch in tqdm(DataLoader(val_set, batch_size = 1, shuffle=False)):
    processed_batch = process_data_to_model_inputs(batch, special_tokens=['[Dominant]', '[Reference]'])
    processed_batch_cuda = {}
    for key in ["input_ids", "attention_mask", "global_attention_mask", "labels"]:
        processed_batch_cuda[key] = torch.tensor(processed_batch[key]).to(device)
    predicted_abstract_ids = model.generate(
        processed_batch_cuda["input_ids"], 
        attention_mask=processed_batch_cuda["attention_mask"], 
        global_attention_mask=processed_batch_cuda["global_attention_mask"]
    )
    out = tokenizer.batch_decode(predicted_abstract_ids, skip_special_tokens=False)
#     print(out)
    target = batch["target"]
    for o, t, part_id, source in zip(out, target, batch["id"], batch["source"]):
        accumulated_data.append(
            {"source": source, "target": t, 
             "generated": o, "part_id": part_id}
        )

In [ ]:
# len(accumulated_data)

In [25]:
reference_predicted = []
reference_reference = []
dominant_predicted = []
dominant_reference = []
accumulated_data = []
i = 0
# with open(os.path.join(path, "sample_output.txt"),"w") as f:
for batch in tqdm(DataLoader(val_set, batch_size = 6, shuffle=False)):
    processed_batch = process_data_to_model_inputs(batch, special_tokens=['[Dominant]', '[Reference]'])
    processed_batch_cuda = {}
    for key in ["input_ids", "attention_mask", "global_attention_mask", "labels"]:
        processed_batch_cuda[key] = torch.tensor(processed_batch[key]).to(device)
    predicted_abstract_ids = model.generate(
        processed_batch_cuda["input_ids"], 
        attention_mask=processed_batch_cuda["attention_mask"], 
        global_attention_mask=processed_batch_cuda["global_attention_mask"]
    )
    out = tokenizer.batch_decode(predicted_abstract_ids, skip_special_tokens=True)
    target = batch["target"]
    for o, t, part_id, source in zip(out, target, batch["id"], batch["source"]):
        accumulated_data.append(
            {"source": source, "target": t, 
             "generated": o, "part_id": part_id}
        )    
    cleaned_out = []
    cleaned_target = []
    for pred, label, source in zip(out, target, batch["source"]):
       for c in get_citations(source):
           pred = pred.replace(c,"")
           label = label.replace(c,"")
       if "[Dominant]" in source:
           dominant_predicted.append(pred)
           dominant_reference.append(label)
       elif "[Reference]" in source:
           reference_predicted.append(pred)
           reference_reference.append(label)
        
    i += 1
    if i % 20 == 0:
        print(rouge.compute(
            predictions=dominant_predicted, 
            references=dominant_reference, 
            rouge_types=["rouge1","rouge2","rougeL"]
        ))
        
        print(rouge.compute(
            predictions=reference_predicted, 
            references=reference_reference, 
            rouge_types=["rouge1","rouge2","rougeL"]
        ))

  4%|▍         | 20/498 [05:43<3:04:08, 23.11s/it]

{'rouge1': AggregateScore(low=Score(precision=0.2620824714924607, recall=0.17616972818852342, fmeasure=0.1985584457245057), mid=Score(precision=0.3212742973526913, recall=0.21263331650133646, fmeasure=0.2399728189214046), high=Score(precision=0.38685212733972396, recall=0.24934519988700207, fmeasure=0.28197728111010084)), 'rouge2': AggregateScore(low=Score(precision=0.09040883367522495, recall=0.05185882077957665, fmeasure=0.06319686566070278), mid=Score(precision=0.13303445061211108, recall=0.07718802087419462, fmeasure=0.09225461959085002), high=Score(precision=0.18755884254365476, recall=0.10826859441603635, fmeasure=0.12762758099771973)), 'rougeL': AggregateScore(low=Score(precision=0.2245291910134432, recall=0.14947390516012257, fmeasure=0.1696338451263147), mid=Score(precision=0.27234959590567065, recall=0.1800089990693695, fmeasure=0.20292593659081282), high=Score(precision=0.3309965572756981, recall=0.21385803550530336, fmeasure=0.24264302517790187))}
{'rouge1': AggregateScore(

  8%|▊         | 40/498 [13:09<2:32:10, 19.94s/it]

{'rouge1': AggregateScore(low=Score(precision=0.267369985331449, recall=0.19214676365740183, fmeasure=0.21189592352827513), mid=Score(precision=0.30109561075470914, recall=0.21322391057897044, fmeasure=0.23598497870154928), high=Score(precision=0.3351888613101005, recall=0.23715076068675678, fmeasure=0.260529217057712)), 'rouge2': AggregateScore(low=Score(precision=0.07925163353353526, recall=0.051474993638085144, fmeasure=0.059566012304695555), mid=Score(precision=0.10210793665058815, recall=0.06626541243069929, fmeasure=0.0762775466155323), high=Score(precision=0.13008556064091073, recall=0.08278740102417265, fmeasure=0.09427889258584592)), 'rougeL': AggregateScore(low=Score(precision=0.224644453456219, recall=0.16081477742354958, fmeasure=0.17871463171064172), mid=Score(precision=0.25129364655335995, recall=0.18076419177975223, fmeasure=0.1984281750623746), high=Score(precision=0.28068767052919097, recall=0.20054455811677657, fmeasure=0.2182232696621422))}
{'rouge1': AggregateScore(

 12%|█▏        | 60/498 [21:23<3:09:43, 25.99s/it]

...

 96%|█████████▌| 479/498 [3:00:22<07:31, 23.75s/it]

{'rouge1': AggregateScore(low=Score(precision=0.280402017344057, recall=0.21519740558427974, fmeasure=0.22467516539985657), mid=Score(precision=0.29060182861785716, recall=0.22336323397079555, fmeasure=0.23247707727901992), high=Score(precision=0.30138310123318396, recall=0.23141170006800768, fmeasure=0.24041304050121465)), 'rouge2': AggregateScore(low=Score(precision=0.08415746414542816, recall=0.06085862953931077, fmeasure=0.06525380428641869), mid=Score(precision=0.09163291354535526, recall=0.06682249616492736, fmeasure=0.07141796932898459), high=Score(precision=0.09950773727520354, recall=0.07291238885038649, fmeasure=0.07752185568169236)), 'rougeL': AggregateScore(low=Score(precision=0.23084990052178156, recall=0.17778038566104917, fmeasure=0.18494229591845274), mid=Score(precision=0.2399791098451084, recall=0.18502672173507287, fmeasure=0.19213393264907108), high=Score(precision=0.24932432945883834, recall=0.19229090015803396, fmeasure=0.19927797618048668))}


 96%|█████████▋| 480/498 [3:00:44<06:57, 23.21s/it]

{'rouge1': AggregateScore(low=Score(precision=0.26521573159484446, recall=0.2782457898148648, fmeasure=0.24935265846905438), mid=Score(precision=0.28236250378297256, recall=0.2961324865693957, fmeasure=0.26465851056972867), high=Score(precision=0.29916905946981465, recall=0.31219264913490635, fmeasure=0.28037296514066323)), 'rouge2': AggregateScore(low=Score(precision=0.10795270868081616, recall=0.11799006237633415, fmeasure=0.10104870576903559), mid=Score(precision=0.12114850319992207, recall=0.13112188869885, fmeasure=0.11314175759121886), high=Score(precision=0.13394715766971887, recall=0.14476639579607897, fmeasure=0.12520037339533643)), 'rougeL': AggregateScore(low=Score(precision=0.26006996961703766, recall=0.2735849198376858, fmeasure=0.24425965712378453), mid=Score(precision=0.27646917203986654, recall=0.29022167044877917, fmeasure=0.25961312479640075), high=Score(precision=0.29311674378183, recall=0.3086318207696876, fmeasure=0.2761188854612854))}


100%|██████████| 498/498 [3:07:11<00:00, 22.55s/it]


In [46]:
# model

In [35]:
# with open(os.path.join("/home/bxm200000/models/led_paragraph_generation_v3/", "sample_output_train_paragraph_gen_v3.2.json"), 'w') as f:
#     json.dump(accumulated_data, f)

In [28]:
len(accumulated_data)

2983

In [123]:
with open(os.path.join("/home/bxm200000/models/led_paragraph_generation_v3/", "sample_output_train_paragraph_gen_v3.2.json"), 'r') as f:
    accumulated_data = json.loads(f.read())

In [110]:
# accumulated_data[0]

In [111]:
path = "/home/bxm200000/models/led_length_control/positional_length_control_para_gen_predict_length_v21.1/checkpoint-32000/"

In [112]:
with open(os.path.abspath(os.path.join(path, "../sample_output.json")), 'r') as f:
    accumulated_data = json.loads(f.read())

In [78]:
ddd = accumulated_data[2]

In [80]:
# ddd

In [124]:
rouge = load_metric("rouge")

In [125]:
def strip_context(paragraph):
    """strip context from paragraph and only return span"""
    return paragraph.split("<context>")[1].split("</context>")[0].strip()

In [126]:
# accumulated_data[0]

In [127]:
new_accumulated_data = accumulated_data

In [128]:
predicted, references = [], []
dominant_predicted, dominant_reference = [], []
reference_predicted, reference_reference = [], []
for index, data in enumerate(new_accumulated_data):
    target = data["target"]
    gen = data["generated"]
    
    if "<context>" in gen and "</context>" in gen:
        
        target = strip_context(data["target"])
        gen = strip_context(data["generated"])
        data["generated"] = gen
        data["target"] = target
        
        new_accumulated_data[index] = data
        # remove citations         
        for c in get_citations(data["source"]):
            c = c.replace(",", "").replace(".", "")
            gen = gen.replace(",", "").replace(".", "").replace(c, "")
            target = target.replace(",", "").replace(".", "").replace(c, "")
        
        predicted.append(gen)
        references.append(target)
        
        if "[Dominant]" in data["source"]:
            dominant_predicted.append(gen)
            dominant_reference.append(target)
            
            
        if "[Reference]" in data["source"]:
            reference_predicted.append(gen)
            reference_reference.append(target)


In [129]:
# plt.hist(
#     [tokenizer.tokenize(target).__len__() - tokenizer.tokenize(predicted).__len__() \
#      for target, predicted in zip(references, predicted)],
#     bins=50,
#     range = (-100, 100)
# )

In [116]:
# with open(os.path.join("/home/bxm200000/models/led_paragraph_generation_v3/", "sample_output_train_paragraph_gen_strip_v3.3.json"), 'w') as f:
#     json.dump(new_accumulated_data, f)

In [117]:
# i =4
# i += 1
# accumulated_data[i]

In [118]:
# rouge.compute(predictions=predicted, references=references, rouge_types=["rouge1","rouge2","rougeL"])

In [130]:
target

' annotated the Luna corpus 3 which does not include English annotations'

In [131]:
rouge.compute(predictions=predicted, references=references, rouge_types=["rouge1","rouge2","rougeL"])

{'rouge1': AggregateScore(low=Score(precision=0.21124460273478848, recall=0.2760019409231681, fmeasure=0.2094705587951958), mid=Score(precision=0.2203567834437479, recall=0.28703975940955084, fmeasure=0.21778941370227095), high=Score(precision=0.22944923376345533, recall=0.29761193585704865, fmeasure=0.2259181729667761)),
 'rouge2': AggregateScore(low=Score(precision=0.0703281742355175, recall=0.09465983653498301, fmeasure=0.06905230442419608), mid=Score(precision=0.07649506718235055, recall=0.10305123449691425, fmeasure=0.07532427184587537), high=Score(precision=0.08320664194208792, recall=0.11191122202367075, fmeasure=0.08166549234124715)),
 'rougeL': AggregateScore(low=Score(precision=0.18616921418624321, recall=0.2469718414930492, fmeasure=0.18610663792328194), mid=Score(precision=0.19460780291508112, recall=0.25635423220379416, fmeasure=0.1934282808580508), high=Score(precision=0.20281944984675335, recall=0.26650836014325663, fmeasure=0.2013999128367323))}

In [132]:
rouge.compute(
    predictions=dominant_predicted, 
    references=dominant_reference, rouge_types=["rouge1","rouge2","rougeL"], use_stemmer=True)

{'rouge1': AggregateScore(low=Score(precision=0.2521972424115864, recall=0.2949464151559301, fmeasure=0.2392433991632979), mid=Score(precision=0.2615801686766283, recall=0.3040527915677015, fmeasure=0.24613630235562173), high=Score(precision=0.2710635573034785, recall=0.31328535049245004, fmeasure=0.25319517445395634)),
 'rouge2': AggregateScore(low=Score(precision=0.061756626805981416, recall=0.070434797507288, fmeasure=0.05733334218023684), mid=Score(precision=0.06727584695058104, recall=0.07625996042020035, fmeasure=0.061773983804685254), high=Score(precision=0.07352286044441975, recall=0.08263460060292332, fmeasure=0.06705751732613527)),
 'rougeL': AggregateScore(low=Score(precision=0.1943844663947913, recall=0.22600995627848006, fmeasure=0.1841215350463397), mid=Score(precision=0.20162014032970038, recall=0.23383721333992896, fmeasure=0.18944829544319963), high=Score(precision=0.209892055603762, recall=0.24197344008203958, fmeasure=0.19534121488215386))}

In [133]:
rouge.compute(
    predictions=reference_predicted, 
    references=reference_reference, 
    rouge_types=["rouge1","rouge2","rougeL"], use_stemmer=True)

{'rouge1': AggregateScore(low=Score(precision=0.2100272539726637, recall=0.3024299313658022, fmeasure=0.21704336614103276), mid=Score(precision=0.22366404914570404, recall=0.32002726857057173, fmeasure=0.22985812269979283), high=Score(precision=0.23856682198580942, recall=0.33709592301252367, fmeasure=0.24432709112879286)),
 'rouge2': AggregateScore(low=Score(precision=0.08400731558117439, recall=0.12337946528710475, fmeasure=0.08573042372339902), mid=Score(precision=0.09405605142475731, recall=0.13695622714200884, fmeasure=0.09548413873057918), high=Score(precision=0.10572322683408894, recall=0.1509369275913055, fmeasure=0.10676721742930914)),
 'rougeL': AggregateScore(low=Score(precision=0.20241219222316614, recall=0.2915125817773235, fmeasure=0.20871017939689857), mid=Score(precision=0.21544506179036507, recall=0.30864203721651046, fmeasure=0.221594975842006), high=Score(precision=0.22959993408938173, recall=0.32431389004718497, fmeasure=0.23492011558701797))}